<a href="https://colab.research.google.com/github/monda00/horse-race-notebook/blob/master/horse_race_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>